In [152]:
import pandas as pd
import numpy as np
import math
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import gc

import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter

# Import libraries
import nltk.translate.chrf_score as chrf
from rouge import Rouge
import nltk
import nltk.translate.gleu_score as gleu

import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')
import nltk.translate.meteor_score as meteor
import nltk.translate.nist_score as nist
import nltk.translate.bleu_score as bleu

#import spacy
from nltk.tokenize import RegexpTokenizer

from sklearn.metrics.pairwise import cosine_similarity
#import itertools

import codecs
import jieba

#Import nltk Snowball stemmer
from nltk.stem.snowball import SnowballStemmer

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from stop_words import get_stop_words

In [3]:
scores_cs = pd.read_csv("corpus/cs-en/scores.csv")
scores_de = pd.read_csv("corpus/de-en/scores.csv")
scores_en_fi = pd.read_csv("corpus/en-fi/scores.csv")
scores_en_zh = pd.read_csv("corpus/en-zh/scores.csv")
scores_ru = pd.read_csv("corpus/ru-en/scores.csv")
scores_zh = pd.read_csv("corpus/zh-en/scores.csv")

In [5]:
# Work with contractions using pycontractes lib that chooses a proper form
# instead ambiguous "you'd" -> "you had / you would"

from pycontractions import Contractions

# Download 'GoogleNews-vectors-negative300.bin' file to a folder with this jupiter notebook
# if needed (size is more 3 Gb)
# archive on - https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
cont = Contractions('GoogleNews-vectors-negative300.bin')

# prevents loading on first expand_texts call
# usually takes time
cont.load_models()

In [6]:
a = scores_de.translation
a[6]

"It's now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work."

In [7]:
b = pd.Series(list(cont.expand_texts(a, precise=True)))

In [8]:
# Check how contracion expanding worked
b[6]

'it is now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.'

In [9]:
# As contraction extractions work really slow
# they were done outside preprocessing function
references_cs = pd.Series(list(cont.expand_texts(scores_cs['reference'], precise=True)))
references_de = pd.Series(list(cont.expand_texts(scores_de['reference'], precise=True)))
references_ru = pd.Series(list(cont.expand_texts(scores_ru['reference'], precise=True)))
references_zh = pd.Series(list(cont.expand_texts(scores_zh['reference'], precise=True)))

In [10]:
# Same for translations
translation_cs = pd.Series(list(cont.expand_texts(scores_cs['translation'], precise=True)))
translation_de = pd.Series(list(cont.expand_texts(scores_de['translation'], precise=True)))
translation_ru = pd.Series(list(cont.expand_texts(scores_ru['translation'], precise=True)))
translation_zh = pd.Series(list(cont.expand_texts(scores_zh['translation'], precise=True)))

In [11]:
# Check how expanding contractions worked
print(scores_de['translation'][6], '   ---->  \n',translation_de[6])

It's now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.    ---->  
 it is now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.


In [12]:
# release references and 'garbage collect' to free memory
import gc
del cont
gc.collect()

98

In [13]:
def preprocessing(series, pontuation = False, zh = False):
    processed_corpus = []
    #stop_words = set(stopwords.words())
    for i in tqdm(range(len(series))):
        text = series[i]
        
        #remove tags
        text = BeautifulSoup(text).get_text()
        
        #Remove punctuations
        if pontuation==True:
            text = re.sub(r'[\s+\.\!\/_,$%^*(+\"\']+|[+——！，？、~@#￥%……&*（）:：；《）《》“”()»〔〕-]+', ' ', text)
        else:
            pass

        #Convert to lowercase
        text = text.lower()
        
        # Convert to list from string
        
        # For en-zh language pair use jieba for words cut
        if zh == True:
            text = jieba.cut_for_search(text)
        else:
            text = text.split()

        #Lemmatisation
        lem = WordNetLemmatizer()
        #text = [lem.lemmatize(word) for word in text if not word in stop_words] 
        text = " ".join(text)
        processed_corpus.append(text)
    return processed_corpus

In [14]:
references_cs = preprocessing(references_cs)
references_de = preprocessing(references_de)
references_en_fi = preprocessing(scores_en_fi['reference'])
references_en_zh = preprocessing(scores_en_zh['reference'], zh = True)
references_ru = preprocessing(references_ru)
references_zh = preprocessing(references_zh)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\murmu\AppData\Local\Temp\jieba.cache
Loading model cost 0.839 seconds.
Prefix dict has been built successfully.


In [15]:
translation_cs = preprocessing(translation_cs)
translation_de = preprocessing(translation_de)
translation_en_fi = preprocessing(scores_en_fi['translation'])
translation_en_zh = preprocessing(scores_en_zh['translation'], zh = True)
translation_ru = preprocessing(translation_ru)
translation_zh = preprocessing(translation_zh)

In [16]:
# METRICS from your file text_mining1

In [17]:
def chrf_metric(translation, references):
    chrf_metric = []
    for i in range(len(translation)):
        row = chrf.sentence_chrf([translation[i]], references[i])
        chrf_metric.append(row)
    return chrf_metric

chf_cs = chrf_metric(translation_cs, references_cs)
scores_cs['chf'] = chf_cs
chf_de = chrf_metric(translation_de, references_de)
scores_de['chf'] = chf_de
chf_en_fi = chrf_metric(translation_en_fi, references_en_fi)
scores_en_fi['chf'] = chf_en_fi
chf_en_zh = chrf_metric(translation_en_zh, references_en_zh)
scores_en_zh['chf'] = chf_en_zh
chf_ru = chrf_metric(translation_ru, references_ru)
scores_ru['chf'] = chf_ru
chf_zh = chrf_metric(translation_zh, references_zh)
scores_zh['chf'] = chf_zh

In [18]:
def gleu_metric(translation, references):
    gleu_metric = []
    for i in range(len(translation)):
        row = gleu.sentence_gleu([str(references[i]).split()], str(translation[i]).split(), min_len=1, max_len=2)
        gleu_metric.append(row)
    return gleu_metric

In [19]:
gleu_cs = gleu_metric(translation_cs, references_cs)
scores_cs['gleu'] = gleu_cs

gleu_de = gleu_metric(translation_de, references_de)
scores_de['gleu'] = gleu_de

gleu_en_fi = gleu_metric(translation_en_fi, references_en_fi)
scores_en_fi['gleu'] = gleu_en_fi

gleu_en_zh = gleu_metric(translation_en_zh, references_en_zh)
scores_en_zh['gleu'] = gleu_en_zh

gleu_ru = gleu_metric(translation_ru, references_ru)
scores_ru['gleu'] = gleu_ru

gleu_zh = gleu_metric(translation_zh, references_zh)
scores_zh['gleu'] = gleu_zh

In [20]:
def meteor_metric(translation, references):
    meteor_metric = []
    for i in range(len(translation)):
        row = (meteor.meteor_score([translation[i]], references[i]))
        meteor_metric.append(row)
    return meteor_metric

In [21]:
meteor_cs = meteor_metric(translation_cs, references_cs)
scores_cs['meteor'] = meteor_cs

meteor_de = meteor_metric(translation_de, references_de)
scores_de['meteor'] = meteor_de

meteor_en_fi = meteor_metric(translation_en_fi, references_en_fi)
scores_en_fi['meteor'] = meteor_en_fi

meteor_en_zh = meteor_metric(translation_en_zh, references_en_zh)
scores_en_zh['meteor'] = meteor_en_zh

meteor_ru = meteor_metric(translation_ru, references_ru)
scores_ru['meteor'] = meteor_ru

meteor_zh = meteor_metric(translation_zh, references_zh)
scores_zh['meteor'] = meteor_zh

In [22]:
def bleu_metric(translation, references):
    bleu_metric = []
    for i in range(len(translation)):
        row = (bleu.sentence_bleu([translation[i]], references[i]))
        bleu_metric.append(row)
    return bleu_metric

bleu_cs = bleu_metric(translation_cs, references_cs)
scores_cs['bleu'] = bleu_cs

bleu_de = bleu_metric(translation_de, references_de)
scores_de['bleu'] = bleu_de

bleu_en_fi = bleu_metric(translation_en_fi, references_en_fi)
scores_en_fi['bleu'] = bleu_en_fi

bleu_en_zh = bleu_metric(translation_en_zh, references_en_zh)
scores_en_zh['bleu'] = bleu_en_zh

bleu_ru = bleu_metric(translation_ru, references_ru)
scores_ru['bleu'] = bleu_ru

bleu_zh = bleu_metric(translation_zh, references_zh)
scores_zh['bleu'] = bleu_zh

In [23]:

def nist_metric(translation, references):
    nist_metric = []
    for i in range(len(translation)):
        row = (nist.sentence_nist([translation[i]], references[i],n=1))
        nist_metric.append(row)
    return nist_metric

In [24]:
nist_cs = nist_metric(translation_cs, references_cs)
scores_cs['nist'] = nist_cs

nist_de = nist_metric(translation_de, references_de)
scores_de['nist'] = nist_de

nist_en_fi = nist_metric(translation_en_fi, references_en_fi)
scores_en_fi['nist'] = nist_en_fi

nist_en_zh = nist_metric(translation_en_zh, references_en_zh)
scores_en_zh['nist'] = nist_en_zh

nist_ru = nist_metric(translation_ru, references_ru)
scores_ru['nist'] = nist_ru

nist_zh = nist_metric(translation_zh, references_zh)
scores_zh['nist'] = nist_zh

In [25]:
def corr(scores):
    chf = scores['z-score'].corr(scores['chf'])
    gleu = scores['z-score'].corr(scores['gleu'])
    meteor = scores['z-score'].corr(scores['meteor'])
    bleu = scores['z-score'].corr(scores['bleu'])
    nist = scores['z-score'].corr(scores['nist'])
    #ribes = scores['z-score'].corr(scores['ribes'])
    corr = [chf, gleu, meteor, bleu, nist]
    return corr

In [26]:
corr_cs = corr(scores_cs)
corr_de = corr(scores_de)
corr_en_fi = corr(scores_en_fi)
corr_en_zh = corr(scores_en_zh)
corr_ru = corr(scores_ru)
corr_zh = corr(scores_zh)

In [27]:
columns = ['chf', 'gleu', 'meteor', 'bleu', 'nist']
data = []
index = ['scores_cs', 'scores_de', 'scores_en_fi', 'scores_en_zh', 'scores_ru', 'scores_zh']
correlation = pd.DataFrame(data, columns=columns, index = index)
correlation.iloc[0] = corr_cs
correlation.iloc[1] = corr_de
correlation.iloc[2] = corr_en_fi
correlation.iloc[3] = corr_en_zh
correlation.iloc[4] = corr_ru
correlation.iloc[5] = corr_zh

In [28]:
# Baseline correlation
correlation

,chf,gleu,meteor,bleu,nist
scores_cs,0.46223,0.427899,0.439964,0.46878,0.334652
scores_de,0.341149,0.310098,0.30805,0.346757,0.227363
scores_en_fi,0.611567,0.494636,0.491475,0.619928,0.423513
scores_en_zh,0.423398,0.449157,0.453092,0.468428,0.432876
scores_ru,0.361418,0.333536,0.336746,0.367581,0.26207
scores_zh,0.341207,0.317924,0.326359,0.351904,0.244659


## Calculate cosine similarities for sentences using word embeddings
 - we use pre-trained word embeddings

In [77]:
# english stopwords
english_stop_words = set(stopwords.words('english')).union(STOP_WORDS)

# chinese stopwords
stopwords_zh = codecs.open('stopwords-zh.txt', 'r', 'utf-8').read().split(',')

# finnish stopwords
stop_words_fi = get_stop_words('finnish')

In [30]:
# delete stopwords, lemmatize if needed

def words_count(text):
    tokenizer = RegexpTokenizer(r'\w+')
    # tokenize text
    tokens = tokenizer.tokenize(text)
   
    # delete stop words
    tokens = list(filter(lambda x: x not in english_stop_words, tokens))
    
    # lemmatize the words WordNetLemmatizer from nlkt
    # this made Pearson correlation lower - so was commented
    
    # tokens = list(map(lambda x: nltk.WordNetLemmatizer().lemmatize(x), tokens))
    
    # With spacy lemmatizer (worked better than nltk)
    # it is commented because it made correlation lower
    # tokens = list(map(lambda x: [token.lemma_ for token in nlp(x)], tokens))
    # tokens = list(itertools.chain(*tokens))
    
    return list(Counter(tokens).most_common())

In [31]:
# delete stopwords, lemmatize if needed

def words_count_zh(text):
#     # Tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    # tokenize text
    tokens = tokenizer.tokenize(text)
    # delete stop words
    tokens = list(filter(lambda x: x not in stopwords_zh, tokens))
    return list(Counter(tokens).most_common())

In [33]:
# Load the pre-trained GloVe (Global Vectors for Word Representation) word vectors.
# For cs, de, ru, zh that translated to english
# 6B tokens, 400K vocab, (300d version).
# Download file from https://www.kaggle.com/thanakomsn/glove6b300dtxt
# to a folder with this jupiter notebook (size 1Gb)
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
    return word_to_vec_map
# Download glove.6B.300d.txt to a folder with this jupiter notebook
word_to_vec_map = read_glove_vecs('glove.6B.300d.txt')

In [34]:
# Chinese word vector
# Download word vectors file from https://www.kaggle.com/kerneler/starter-sgns-merge-word-18e5b7b5-9/execution?select=sgns.merge.word
# file to a folder with this jupiter notebook (size 3.5Gb)
word_to_vec_zh = read_glove_vecs('sgns.merge.word')

In [90]:
# We could not find glove nodel for finnish,
# so will work with finnish language separately

In [91]:
# Create column for calculating cosine similarity
scores_cs['Cosine_similarity'], scores_de['Cosine_similarity'] = 0, 0
scores_ru['Cosine_similarity'], scores_zh['Cosine_similarity'] = 0, 0
scores_en_fi['Cosine_similarity'], scores_en_zh['Cosine_similarity'] = 0, 0

In [112]:
# Create lists with languages to work with
ref_list = [references_cs, references_de, references_ru, references_zh, references_en_zh]
trans_list = [translation_cs, translation_de, translation_ru, translation_zh, translation_en_zh]
scores = [scores_cs, scores_de, scores_ru, scores_zh, scores_en_zh]

In [113]:
# Check cosine similarity between two sentences (reference and translation)
# Use different pre-trained word embeddings:
# GloVe - for cs, de, ru, zh that translated to english
# 'sgns.merge.word' - for en-zh pair

for doc in range(len(scores)):
    ref_embeddings = np.zeros((len(scores[doc]),300))
    tr_embeddings = np.zeros((len(scores[doc]),300))
    
    norms_doc_embeddings_tr = np.zeros((len(trans_list[doc]),1))
    norms_doc_embeddings_ref = np.zeros((len(ref_list[doc]),1))
    
    Cosine_similarity = []
    
    for i in tqdm(range(len(trans_list[doc]))):
        if doc == 4:
            words_freq = words_count_zh(trans_list[doc][i])
            doc_embeddings = np.zeros(word_to_vec_zh["a"].shape)
        
        else:
            words_freq = words_count(trans_list[doc][i])
            doc_embeddings = np.zeros(word_to_vec_map["a"].shape)
        num_words = 0
        for word_freq in words_freq:
            word = word_freq[0]
            freq = word_freq[1]
            try:
                #adding word embeddings for each word in the document
                if doc == 4:
                    doc_embeddings += (word_to_vec_zh[word] * freq)
                else:
                    doc_embeddings += (word_to_vec_map[word] * freq)
                num_words += freq
            except:
                continue
        try:
            # doing average
            doc_embeddings /= num_words
        except:
            print("divide by zero encountered for article at index "+str(i))
            continue
        norms_doc_embeddings_tr[i,:] = np.sqrt(np.dot(doc_embeddings,doc_embeddings))
        tr_embeddings[i,:] = doc_embeddings
        
        
        if doc == 4:
            words_freq = words_count_zh(ref_list[doc][i])
            doc_embeddings = np.zeros(word_to_vec_zh["a"].shape)
        else:
            words_freq = words_count(ref_list[doc][i])
            doc_embeddings = np.zeros(word_to_vec_map["a"].shape)
        num_words = 0
        for word_freq in words_freq:
            word = word_freq[0]
            freq = word_freq[1]
            try:
                #adding word embeddings for each word in the document
                if doc == 4:
                    doc_embeddings += (word_to_vec_zh[word] * freq)
                
                else:
                    doc_embeddings += (word_to_vec_map[word] * freq)
                num_words += freq
            except:
                continue
        try:
            # doing average
            doc_embeddings /= num_words
        except:
            print("divide by zero encountered for article at index "+str(i))
            continue
        norms_doc_embeddings_ref[i,:] = np.sqrt(np.dot(doc_embeddings,doc_embeddings))
        ref_embeddings[i,:] = doc_embeddings
    
        # Calculate cosine similarity using dot product of two vectors
        # and norms of vectors
        
        #  The cosine similarity depends on the angle between  vectors .
        #  If  vectors  are very similar, their cosine similarity will be close to 1
        norm_prods = norms_doc_embeddings_ref[i] * norms_doc_embeddings_tr[i]
        dot_prod = np.dot(ref_embeddings[i], tr_embeddings[i].reshape(300,1))
        cos_similarity = dot_prod / norm_prods
        
        Cosine_similarity.append(float(cos_similarity))
    
    scores[doc]['Cosine_similarity'] = pd.Series(Cosine_similarity)

In [114]:
def corr_new(scores):
    chf = scores['z-score'].corr(scores['chf'])
    gleu = scores['z-score'].corr(scores['gleu'])
    meteor = scores['z-score'].corr(scores['meteor'])
    bleu = scores['z-score'].corr(scores['bleu'])
    nist = scores['z-score'].corr(scores['nist'])
    cos = scores['z-score'].corr(scores['Cosine_similarity'])
    corr = [chf, gleu, meteor, bleu, nist, cos]
    return corr

In [115]:
corr_cs_new = corr_new(scores_cs)
corr_de_new = corr_new(scores_de)
corr_ru_new = corr_new(scores_ru)
corr_zh_new = corr_new(scores_zh)

corr_en_zh_new = corr_new(scores_en_zh)

In [116]:
columns = ['chf', 'gleu', 'meteor', 'bleu', 'nist', 'Cos_sim']
data = []
index = ['scores_cs', 'scores_de',  'scores_en_zh', 'scores_ru', 'scores_zh']
correlation = pd.DataFrame(data, columns=columns, index = index)
correlation.iloc[0] = corr_cs_new
correlation.iloc[1] = corr_de_new
correlation.iloc[2] = corr_en_zh_new

correlation.iloc[3] = corr_ru_new
correlation.iloc[4] = corr_zh_new

In [117]:
# Calculate correlation between z-score and cosine_similarity columns
# for all language pairs except finnish
correlation

,chf,gleu,meteor,bleu,nist,Cos_sim
scores_cs,0.46223,0.427899,0.439964,0.46878,0.334652,0.368188
scores_de,0.341149,0.310098,0.30805,0.346757,0.227363,0.290548
scores_en_zh,0.423398,0.449157,0.453092,0.468428,0.432876,0.360268
scores_ru,0.361418,0.333536,0.336746,0.367581,0.26207,0.304984
scores_zh,0.341207,0.317924,0.326359,0.351904,0.244659,0.27196


### Fiinish language

In [136]:
# Download pre-trained multilingual BERT model 
# to get sentence embeddings
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained("bert-base-multilingual-cased")

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [144]:
text = "Long text about stars."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

In [155]:
normalize_a = tf.nn.l2_normalize(cls_embedding,0)        
normalize_b = tf.nn.l2_normalize(cls_embedding,0)
cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
float(cos_similarity)

1.0000001192092896

In [163]:
# Attention: this cell takes more than hour to calculate cosine similaritites

# In order to compare the embeddings of sentences with  pretrained BERT 
# we use the value of the CLS token that corresponds to the first token of the output
scores_en_fi['Cosine_similarity'] = 0
Cosine = []
for doc in tqdm(range(len(scores_en_fi))):
    
    # text from references_en_fi and translation_en_fi
    text1 = references_en_fi[doc]
    text2 = translation_en_fi[doc]
    
    encoded_input1 = tokenizer(text1, return_tensors='tf')
    output1 = model(encoded_input1)
    
    encoded_input2 = tokenizer(text2, return_tensors='tf')
    output2 = model(encoded_input2)
    
    last_hidden_states_1 = output1[0]
    cls_embedding_1 = last_hidden_states_1[0][0]   
    last_hidden_states_2 = output2[0]
    cls_embedding_2 = last_hidden_states_2[0][0]
    normalize_a = tf.nn.l2_normalize(cls_embedding_1,0)        
    normalize_b = tf.nn.l2_normalize(cls_embedding_2,0)
    cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
    Cosine.append(float(cos_similarity))
    
scores_en_fi['Cosine_similarity'] = pd.Series(Cosine)

In [165]:
columns = ['chf', 'gleu', 'meteor', 'bleu', 'nist', 'Cos_sim']
data = []
index = ['scores_cs', 'scores_de', 'scores_en_fi', 'scores_en_zh', 'scores_ru', 'scores_zh']
correlation = pd.DataFrame(data, columns=columns, index = index)

# Recalculate only en-fi score
corr_en_fi_new = corr_new(scores_en_fi)

correlation.iloc[0] = corr_cs_new
correlation.iloc[1] = corr_de_new


correlation.iloc[2] = corr_en_fi_new
correlation.iloc[3] = corr_en_zh_new

correlation.iloc[4] = corr_ru_new
correlation.iloc[5] = corr_zh_new

In [166]:
# Let's check our Pearson correlation coefficients
correlation

,chf,gleu,meteor,bleu,nist,Cos_sim
scores_cs,0.46223,0.427899,0.439964,0.46878,0.334652,0.368188
scores_de,0.341149,0.310098,0.30805,0.346757,0.227363,0.290548
scores_en_fi,0.611567,0.494636,0.491475,0.619928,0.423513,0.150627
scores_en_zh,0.423398,0.449157,0.453092,0.468428,0.432876,0.360268
scores_ru,0.361418,0.333536,0.336746,0.367581,0.26207,0.304984
scores_zh,0.341207,0.317924,0.326359,0.351904,0.244659,0.27196


In [ ]:
# We will pre-process finish docs, trying to increase our correlation with z-score

In [ ]:
# Attention: this library can have some isuues during installation
# Some advices how to resolve them from offical page - https://voikko.puimula.org/python.html

#Import the Voikko to work with finnish lemmas
import libvoikko

# Check how this library works:

#Define a Voikko class for Finnish
v = libvoikko.Voikko(u"fi")

#A word that might or might not be in base form
word = "lehmähauteen"

#Analyze the word
voikko_dict = v.analyze(word)

#Extract the base form as
#analyze() function returns various info for the word
word_baseform = voikko_dict[0]['BASEFORM']

#Print the base form of the word
print(word_baseform)

In [ ]:
# delete stopwords, lemmatize if needed
def preprocess_fi(alltext):
    processed_corpus = []
    #stop_words = set(stopwords.words())
    for i in tqdm(range(len(alltext))):
        text = alltext[i]
        # Tokenize
        tokenizer = RegexpTokenizer(r'\w+')
        # tokenize text
        tokens = tokenizer.tokenize(text)
        temp = []
        #Define a Voikko class for Finnish
        v = libvoikko.Voikko(u"fi")

        # Change words to lemmas
        for word in tokens:
            voikko_dict = v.analyze(word)
            try:
                t = voikko_dict[0]['BASEFORM']
                temp.append(t)
            except:
                temp.append(word)
                continue
        tokens = temp
        # delete stop words
        tokens = list(filter(lambda x: x not in stop_words_fi, tokens))
        #Convert to lowercase because Voikko returns some words that begin from upper case
        text = text.lower()
        text = " ".join(tokens)
        processed_corpus.append(text)
    return processed_corpus

In [ ]:
# Pre-process finnish sentences (references and translations using preprocess_fi)
references_en_fi_new = preprocess_fi(references_en_fi)
translation_en_fi_new = preprocess_fi(translation_en_fi)

In [168]:
# Attention: this cell takes more than hour to calculate cosine similaritites

# In order to compare the embeddings of sentences with  pretrained BERT 
# we use the value of the CLS token that corresponds to the first token of the output
scores_en_fi['Cosine_similarity'] = 0
Cosine = []
for doc in tqdm(range(len(scores_en_fi))):
    
    # text from references_en_fi and translation_en_fi
    text1 = references_en_fi_new[doc]
    text2 = translation_en_fi_new[doc]
    
    encoded_input1 = tokenizer(text1, return_tensors='tf')
    output1 = model(encoded_input1)
    
    encoded_input2 = tokenizer(text2, return_tensors='tf')
    output2 = model(encoded_input2)
    
    last_hidden_states_1 = output1[0]
    cls_embedding_1 = last_hidden_states_1[0][0]   
    last_hidden_states_2 = output2[0]
    cls_embedding_2 = last_hidden_states_2[0][0]
    normalize_a = tf.nn.l2_normalize(cls_embedding_1,0)        
    normalize_b = tf.nn.l2_normalize(cls_embedding_2,0)
    cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
    Cosine.append(float(cos_similarity))
    
scores_en_fi['Cosine_similarity'] = pd.Series(Cosine)

In [169]:
columns = ['chf', 'gleu', 'meteor', 'bleu', 'nist', 'Cos_sim']
data = []
index = ['scores_cs', 'scores_de', 'scores_en_fi', 'scores_en_zh', 'scores_ru', 'scores_zh']
correlation = pd.DataFrame(data, columns=columns, index = index)
# Recalculate only en-fi score
corr_en_fi_new = corr_new(scores_en_fi)

correlation.iloc[0] = corr_cs_new
correlation.iloc[1] = corr_de_new

correlation.iloc[2] = corr_en_fi_new
correlation.iloc[3] = corr_en_zh_new

correlation.iloc[4] = corr_ru_new
correlation.iloc[5] = corr_zh_new

In [170]:
# Let's check our Pearson correlation coefficients
correlation

,chf,gleu,meteor,bleu,nist,Cos_sim
scores_cs,0.46223,0.427899,0.439964,0.46878,0.334652,0.368188
scores_de,0.341149,0.310098,0.30805,0.346757,0.227363,0.290548
scores_en_fi,0.611567,0.494636,0.491475,0.619928,0.423513,0.161529
scores_en_zh,0.423398,0.449157,0.453092,0.468428,0.432876,0.360268
scores_ru,0.361418,0.333536,0.336746,0.367581,0.26207,0.304984
scores_zh,0.341207,0.317924,0.326359,0.351904,0.244659,0.27196


In [ ]:
# We increased to 0.161 from 0.150, but still a room for improvment - we will try to work with finBERT
# BERT model that is trained and fine-tuned for finnish